# TPS JULY 2021

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
import plotly.graph_objects as go
import plotly.express as px
import optuna
from ml_metrics import rmsle
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, MinMaxScaler

In [ ]:
train_set = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv",index_col = 0, parse_dates=['date_time'])
test_set = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv",index_col = 0, parse_dates=['date_time'])
sample_submission = pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv")

train = train_set.copy()
test = test_set.copy()
submission = sample_submission.copy()

In [ ]:
train.describe().T.style.background_gradient(subset = ['count'], cmap = 'viridis') \
    .bar(subset = ['mean', '50%']) \
    .bar(subset = ['std'])

In [ ]:
test.describe().T.style.background_gradient(subset = ['count'], cmap = 'viridis') \
    .bar(subset = ['mean', '50%']) \
    .bar(subset = ['std'])

In [ ]:
train.head(3)

In [ ]:
train.tail(3)

In [ ]:
test.head(3)

In [ ]:
test.tail(3)

Assigning target columns to use them later.

In [ ]:
target_1 = train.iloc[:,-3]
target_2 = train.iloc[:,-2]
target_3 = train.iloc[:,-1]

This problem should be considered as seasonal so, focusing on months than year would be beneficial to make predictions. Also, we don't have enough time range as years in dataset. Except the last row, all entries belong to 2010 in training set.

I'm getting month, day and hour attributes and using them as new features. Because pollution can be considered to be related with month, day and hour.

In [ ]:
def mdh(df):
    df["month"] = df.index.month
    df["day"] = df.index.day
    df["hour"] = df.index.hour
    return df

train = mdh(train)
test = mdh(test)
train.head()

In [ ]:
train.tail()

In [ ]:
cols = ["deg_C","relative_humidity","absolute_humidity","sensor_1","sensor_2","sensor_3","sensor_4","sensor_5","target_carbon_monoxide","target_benzene","target_nitrogen_oxides"]
train[cols].plot(figsize = (12, 25), subplots = True, linewidth = 0.6)
plt.show()

### The features of enviroment

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(x=train["deg_C"], name="Train"))

fig.add_trace(go.Box(x=test["deg_C"], name="Test"))
fig.update_layout(xaxis_title_text = 'Degree (C)',template='ggplot2')

Distributions of temperatures in datasets look different from each other. Let's observe this features in another way. Box plots didn't help much.

In [ ]:
months = train.groupby("month",as_index=False).mean().sort_values(by="month", ascending=True)
months_test = test.groupby("month",as_index=False).mean().sort_values(by="month", ascending=True)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=months["month"], y=months["deg_C"],
    name='Train',
    mode='markers'))

fig.add_trace(go.Scatter(
    x=months["month"], y=months_test["deg_C"],
    name='Test'))

fig.update_traces(mode='markers', marker_line_width=1, marker_size=7)
fig.update_layout(title='Monthly Mean', template="ggplot2",
                  xaxis_title="Months",
                  yaxis_title="Degree (C)",
                  yaxis_zeroline=False, xaxis_zeroline=False)


fig.show()

This plot shows difference between box plots above. Monthly mean of temperatures are parallel but obviously 2011's 3rd, 4th and 5th (March, April, May) months are colder that previous years.

There is another thing that is obvious which is there is no record for 2nd month in both 2 years. There can be many reasons for that such as annual maintenance for sensors.

In [ ]:
corr = train[cols].corr()
fig = px.imshow(corr)
fig.show()

Insights:
* Degree, absolute humidity and relative humidity inputs have no correlation with our targets. But that doesn't mean that they don't have correlation at all, there may be nonlinear relation among these parameters.

Sensor correlations with targets:
* Sensor 1 : CO **0.857**, benzene **0.842**, nitrogen oxides **0.643**.
* Sensor 2 : benzene **0.964**, CO **0.821**, nitrogen oxides **0.604**.
* Sensor 3 : benzene **-0.735**, CO **-0.547**, nitrogen oxides **-0.427**.
* Sensor 4 : benzene **0.783**, CO **0.571**, nitrogen oxides **0.235**.
* Sensor 5 : benzene **0.877**, CO **0.834**, nitrogen oxides **0.708**.


# Model

In [ ]:
X = train.drop(["target_carbon_monoxide","target_benzene","target_nitrogen_oxides"], axis=1).values
rscaler = RobustScaler()
X = rscaler.fit_transform(X)
test = rscaler.transform(test)

In [ ]:
def objective(trial,X,y):
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=17)
    params = {
        'reg_alpha' : trial.suggest_loguniform('reg_alpha' , 0.1 , 10),
        'reg_lambda' : trial.suggest_loguniform('reg_lambda' , 0.5 , 50),
        'num_leaves' : trial.suggest_int('num_leaves' , 100 , 500), 
        'learning_rate' : trial.suggest_float('learning_rate' , 0.01 , 0.6),
        'max_depth' : trial.suggest_int('max_depth' , 2 , 10),               
        'n_estimators' : trial.suggest_int('n_estimators' , 1 ,5000),
        'subsample' : trial.suggest_float('subsample' , 0.1 , 1.0),
    }
    lgb = LGBMRegressor(**params)  
    lgb.fit(X_train,y_train,eval_set=[(X_val,y_val)], eval_metric="RMSLE", early_stopping_rounds=10, verbose=False)
        
    y_pred = lgb.predict(X_val)
 
    rms_e = rmsle(y_val, y_pred)
    
    return rms_e

## Optuna

In [ ]:
study1 = optuna.create_study(direction='minimize')
study1.optimize(lambda trial: objective(trial,X,target_1), n_trials=100)
best_params_1 = study1.best_trial.params
print('Number of finished trials:', len(study1.trials))
print('Best trial:', best_params_1)

In [ ]:
study2 = optuna.create_study(direction='minimize')
study2.optimize(lambda trial: objective(trial,X,target_2), n_trials=100)
best_params_2 = study2.best_trial.params
print('Number of finished trials:', len(study2.trials))
print('Best trial:', best_params_2)

In [ ]:
study3 = optuna.create_study(direction='minimize')
study3.optimize(lambda trial: objective(trial,X,target_3), n_trials=100)
best_params_3 = study3.best_trial.params
print('Number of finished trials:', len(study3.trials))
print('Best trial:', best_params_3)

# Fitting

In [ ]:
lgb1 = LGBMRegressor(**best_params_1)
lgb1.fit(X, target_1)

In [ ]:
pred_1 = lgb1.predict(test)
pred_1

In [ ]:
lgb2 = LGBMRegressor(**best_params_2)
lgb2.fit(X, target_2)

In [ ]:
pred_2 = lgb2.predict(test)
pred_2

In [ ]:
lgb3 = LGBMRegressor(**best_params_3)
lgb3.fit(X, target_3)

In [ ]:
pred_3 = lgb3.predict(test)
pred_3

In [ ]:
submission['target_carbon_monoxide'] = pred_1
submission['target_benzene'] = pred_2
submission['target_nitrogen_oxides'] = pred_3
submission.to_csv('submission.csv', index=False)

*If you find this notebook useful, please upvote* 😊